# KURULUM VE BAĞLANTI (SETUP)

In [ ]:
# 📅 Dashboard Geliştirme & Performans Optimizasyonu
# 🎯 Hedef: Looker Studio için yüksek performanslı "Materialized" tablolar oluşturma
# ------------------------------------------------------------------------------

print("🚀 DASHBOARD OPTİMİZASYON SÜRECİ BAŞLIYOR...")
print("="*60)

# 1. Gerekli Kütüphanelerin Kurulumu
!pip install google-cloud-bigquery pandas db-dtypes -q
print("✅ Kütüphaneler yüklendi.")

# 2. Authentication (Kimlik Doğrulama)
from google.colab import auth
auth.authenticate_user()
print("✅ Google Kimlik Doğrulama başarılı.")

# 3. BigQuery Client Ayarları
from google.cloud import bigquery

PROJECT_ID = "YOUR_PROJECT_ID"
DATASET_ID = "milano_mobile_2013"

client = bigquery.Client(project=PROJECT_ID)
print(f"✅ BigQuery Bağlantısı Hazır: {PROJECT_ID}.{DATASET_ID}")

## DASHBOARD PERFORMANS OPTİMİZASYONU

In [ ]:
def create_split_optimized_tables():
    print("\n⚡ DASHBOARD OPTİMİZASYONU: TABLOLAR AYRILIYOR...")
    print("="*60)

    # ---------------------------------------------------------
    # 1. HARİTA İÇİN TABLO (Daily Aggregation + Geometry)
    # ---------------------------------------------------------
    # Hedef: ~70.000 satır (10k hücre x 7 gün) - Çok hızlı çalışır

    print("🗺️ 1. Harita tablosu oluşturuluyor (dashboard_map_daily)...")

    query_map = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.dashboard_map_daily`
    PARTITION BY date
    CLUSTER BY CellID
    AS
    SELECT
        DATE(t.datetime) as date,
        t.CellID,
        -- Geometriyi burada tutuyoruz (Harita için şart)
        ST_GEOGFROMTEXT(ANY_VALUE(g.geometry)) as geometry,

        -- Metrikleri gün bazında topluyoruz
        SUM(t.smsin_winsorized + t.smsout_clean + t.callin_clean + t.callout_clean + t.internet_winsorized) as total_activity,

        -- Etiketler
        ANY_VALUE(
            CASE
                WHEN c.Cluster = 0 THEN 'Konut/Banliyö'
                WHEN c.Cluster = 1 THEN 'İş Merkezi'
                WHEN c.Cluster = 2 THEN 'Karma/Eğlence'
                ELSE 'Diğer'
            END
        ) as land_use_label

    FROM `{PROJECT_ID}.{DATASET_ID}.traffic_winsorized` t
    LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.grid_locations` g ON t.CellID = g.CellID
    LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.analysis_land_use_clusters` c ON t.CellID = c.CellID
    GROUP BY date, CellID
    """

    try:
        client.query(query_map).result()
        table = client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dashboard_map_daily")
        print(f"   ✅ Harita Tablosu Hazır: {table.num_rows:,} satır ({table.num_bytes / (1024*1024):.2f} MB)")
    except Exception as e:
        print(f"   ❌ Harita tablosu hatası: {e}")

    # ---------------------------------------------------------
    # 2. GRAFİKLER İÇİN TABLO (Hourly Data - NO Geometry)
    # ---------------------------------------------------------
    # Hedef: Geometri yükü olmayan saf zaman serisi. Çok hızlıdır.

    print("\n📈 2. Grafik tablosu oluşturuluyor (dashboard_charts_hourly)...")

    query_charts = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.dashboard_charts_hourly`
    PARTITION BY DATE(datetime)
    CLUSTER BY land_use_label
    AS
    SELECT
        TIMESTAMP_TRUNC(t.datetime, HOUR) as datetime,

        -- Kümeleme Sonuçları (Gruplama için)
        CASE
            WHEN c.Cluster = 0 THEN 'Konut/Banliyö'
            WHEN c.Cluster = 1 THEN 'İş Merkezi'
            WHEN c.Cluster = 2 THEN 'Karma/Eğlence'
            ELSE 'Diğer'
        END as land_use_label,

        -- Toplam Metrikler
        SUM(t.smsin_winsorized + t.smsout_clean + t.callin_clean + t.callout_clean + t.internet_winsorized) as total_activity,
        SUM(t.internet_winsorized) as internet_traffic,
        SUM(t.smsin_winsorized + t.smsout_clean) as total_sms,
        SUM(t.callin_clean + t.callout_clean) as total_calls,

        -- Kaç hücreden geldiği
        COUNT(DISTINCT t.CellID) as cell_count

    FROM `{PROJECT_ID}.{DATASET_ID}.traffic_winsorized` t
    LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.analysis_land_use_clusters` c ON t.CellID = c.CellID
    GROUP BY 1, 2
    """

    try:
        client.query(query_charts).result()
        table = client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dashboard_charts_hourly")
        print(f"   ✅ Grafik Tablosu Hazır: {table.num_rows:,} satır ({table.num_bytes / (1024*1024):.2f} MB)")
    except Exception as e:
        print(f"   ❌ Grafik tablosu hatası: {e}")

    print("\n✨ OPTİMİZASYON TAMAMLANDI!")

# Çalıştır
create_split_optimized_tables()

## DOĞRULAMA (SANITY CHECK)

In [ ]:
def verify_split_tables():
    print("🔍 YENİ TABLOLARIN KONTROLÜ")
    print("="*60)

    tables = ['dashboard_map_daily', 'dashboard_charts_hourly']

    for table_name in tables:
        try:
            table_ref = client.get_table(f"{PROJECT_ID}.{DATASET_ID}.{table_name}")
            print(f"\n📋 TABLO: {table_name}")
            print(f"   📊 Satır Sayısı: {table_ref.num_rows:,}")
            print(f"   💾 Boyut: {table_ref.num_bytes / (1024*1024):.2f} MB")

            # Sütunları listele (Özellikle Geometry kontrolü için)
            print("   🗂️ Sütunlar:")
            for schema in table_ref.schema:
                print(f"      - {schema.name} ({schema.field_type})")

        except Exception as e:
            print(f"❌ Hata ({table_name}): {e}")

verify_split_tables()